In [1]:
import urllib.request
import gzip
import json
import time
import datetime

In [2]:
submissions = []

start = 1514540441
while True:
    url = 'https://kenkoooo.com/atcoder/atcoder-api/v3/user/submissions?user=goropikari&from_second=' + str(start)
    req = urllib.request.Request(
        url=url,
        headers = {'Accept-Encoding': 'gzip', 'Content-Type': 'application/json'}
    )
    with urllib.request.urlopen(req) as f:
        data = f.read()
    
    res = json.loads(gzip.decompress(data).decode('utf8'))
    if len(res) == 0:
        break
    submissions.extend(res)

    start = submissions[-1]['epoch_second']+1
    print(start)
    time.sleep(1)

with open('submissions.json', 'w') as f:
    json.dump(submissions, f)

1543063135
1596259640
1613218014
1641340433
1646494648
1734774393
1738486483
1740816894


In [14]:
problem_url = 'https://kenkoooo.com/atcoder/resources/problem-models.json'
req = urllib.request.Request(
        url=problem_url,
        headers = {'Accept-Encoding': 'gzip', 'Content-Type': 'application/json'}
    )
with urllib.request.urlopen(req) as f:
    data = f.read()

problems = res = json.loads(gzip.decompress(data).decode('utf8'))

with open('problem-models.json', 'w') as f:
    json.dump(problems, f)

In [4]:
def to_jst(unixtime):
    return datetime.datetime.fromtimestamp(
        unixtime, 
        datetime.timezone(datetime.timedelta(hours=9))
    ).strftime('%Y%m%d')

def difficulty_color(x):
    if x < 400:
        return 'gray'
    if x < 800:
        return 'brown'
    if x < 1200:
        return 'green'
    if x < 1600:
        return 'cyan'
    if x < 2000:
        return 'blue'
    if x < 2400:
        return 'yellow'
    if x < 2800:
        return 'orange'
    return 'red'

In [5]:
solved_problem = set()
freq = {}
start_date = to_jst(submissions[0]['epoch_second'])
end_date = to_jst(submissions[-1]['epoch_second'])

ndt = datetime.datetime.strptime(start_date, '%Y%m%d')
edt = datetime.datetime.strptime(end_date, '%Y%m%d')
while ndt <= edt:
    s = ndt.strftime('%Y%m%d')
    freq[s] = {
        'gray': 0,
        'brown': 0,
        'green': 0,
        'cyan': 0,
        'blue': 0,
        'yellow': 0,
        'orange': 0,
        'red': 0,
    }
    ndt += datetime.timedelta(days=1)

In [6]:
for sub in submissions:
    if sub['result'] != 'AC':
        continue
    problem_id = sub['problem_id']
    if problem_id in solved_problem:
        continue
    try:
        solved_problem.add(problem_id)
        diff = problems[problem_id]['difficulty']
        if difficulty_color(diff) == 'red':
            print(problem_id)
            print(problems[problem_id])
        freq[to_jst(sub['epoch_second'])][difficulty_color(diff)] += 1
    except:
        pass

In [7]:
start_date = to_jst(submissions[0]['epoch_second'])
end_date = to_jst(submissions[-1]['epoch_second'])

ndt = datetime.datetime.strptime(start_date, '%Y%m%d')
edt = datetime.datetime.strptime(end_date, '%Y%m%d')
while ndt < edt:
    s = ndt.strftime('%Y%m%d')
    prev = freq[s]
    ndt += datetime.timedelta(days=1)
    ns = ndt.strftime('%Y%m%d')
    for k,v in freq[s].items():
        freq[ns][k] += v

In [13]:
with open('stat.json', 'w') as f:
    json.dump(freq, f)

In [2]:
import json
with open('stat.json', 'r') as f:
    freq = json.load(f)

In [14]:
modfreq = dict()
keys = list(freq.keys())
keys.sort()
prev = None
for key in keys:
    if prev != freq[key]:
        prev = freq[key]        
        modfreq[key] = freq[key]

In [17]:
len(modfreq), len(freq)

(400, 2624)